In [1]:
import polars as pl
import numpy as np

In [2]:
# Load the data
data0  =  pl.read_parquet("data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=6/part-0.parquet")
data1  =  pl.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=7/part-0.parquet')
data2  =  pl.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=8/part-0.parquet')
data3  =  pl.read_parquet('data/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=9/part-0.parquet')

In [3]:
train_data = pl.concat([data0, data1,data2,data3])  # 3 partitions
del data0 ,data1, data2, data3 # free up memory

In [4]:
train_data = train_data.with_columns([
    pl.col(column).fill_null(pl.col(column).mean()) for column in train_data.columns
])

In [5]:
# define the columns
lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]

In [6]:
#new lag data
lags = train_data.select(lag_cols_original)
lags = lags.rename({ f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)})

lags = lags.with_columns((pl.col("date_id") + 1).alias("date_id"))
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last() # pick up last record of previous date
train_data = train_data.join(lags, on=["date_id", "symbol_id"], how="left")
del lags ,lag_cols_original
train_data = train_data.fill_null(0)

In [7]:
train_data.write_parquet('data/df.parquet')

In [8]:
num_val = 1670
val_set = train_data.filter(pl.col("date_id") >= num_val)
val_set.write_parquet('data/validate_data.parquet')
train_set = train_data.filter(pl.col("date_id") < num_val)
train_set.write_parquet('data/train_data.parquet')